In [15]:
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.ion()
import pandas as pd
from scipy import interpolate
from scipy import stats
from datetime import datetime, timedelta
import pytz
from coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects
import geopandas as gpd
import tqdm.auto as tqdm
from glob import glob

In [22]:
files = sorted(glob("shoreline_data_run6/nzd*/time_series_tidally_corrected.csv"))
len(files)
pd.read_csv(files[0]).tail()

,Unnamed: 0,dates,nzd0001-0000,nzd0001-0001,nzd0001-0002,nzd0001-0003,nzd0001-0004,nzd0001-0005,nzd0001-0006,nzd0001-0007,...,nzd0001-0023,nzd0001-0024,nzd0001-0025,nzd0001-0026,nzd0001-0027,nzd0001-0028,nzd0001-0029,nzd0001-0030,nzd0001-0031,nzd0001-0032
177,177,2021-05-01 22:17:07+00:00,174.702851,NaN,178.023459,191.815325,204.759809,NaN,245.847728,265.100128,...,252.020917,248.925421,248.447457,253.193736,250.380017,259.676887,254.461642,255.974983,254.995313,260.239137
178,178,2021-05-17 22:17:12+00:00,NaN,NaN,NaN,NaN,NaN,NaN,234.706588,245.093471,...,226.345775,225.327493,222.358132,229.342560,237.483581,243.185903,238.876358,230.442660,228.176415,237.143330
179,179,2021-06-18 22:17:27+00:00,NaN,NaN,185.730259,NaN,196.514209,217.74784,234.805771,224.857692,...,211.269169,210.184038,212.404057,217.483880,224.446490,244.339407,240.928149,247.393279,247.946689,259.091928
180,180,2021-07-04 22:17:29+00:00,180.263301,NaN,177.743235,NaN,194.454097,NaN,239.352236,NaN,...,225.452269,223.574190,223.746108,231.444424,240.131786,245.887173,246.759335,252.158740,250.708964,259.146114
181,181,2021-09-22 22:17:53+00:00,NaN,NaN,163.745652,NaN,189.369697,NaN,225.546755,240.039959,...,212.920671,206.178345,201.300026,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
poly = gpd.read_file("polygons.geojson")
poly = poly[poly.id.str.startswith("nzd")]
poly.set_index("id", inplace=True)
poly

,area,id_sorted,northing,geometry
id,,,,
nzd0001,3.069093e+06,nzd0001,-4.085604e+06,"POLYGON ((172.96406 -34.43054, 172.99324 -34.4..."
nzd0002,1.507931e+06,nzd0002,-4.086100e+06,"POLYGON ((172.93456 -34.42857, 172.95856 -34.4..."
nzd0003,2.484730e+06,nzd0003,-4.087232e+06,"POLYGON ((173.00980 -34.42055, 173.00088 -34.4..."
nzd0006,9.619509e+05,nzd0004,-4.090732e+06,"POLYGON ((173.00532 -34.46694, 173.00612 -34.4..."
nzd0007,3.109433e+06,nzd0005,-4.095516e+06,"POLYGON ((172.99847 -34.48102, 173.00081 -34.5..."
...,...,...,...,...
nzd0317,9.986110e+05,nzd0556,-4.971830e+06,"POLYGON ((172.32302 -40.71870, 172.31210 -40.7..."
nzd0316,4.300058e+06,nzd0557,-4.966165e+06,"POLYGON ((172.37076 -40.70229, 172.39571 -40.6..."
nzd0313,2.291974e+07,nzd0558,-4.939929e+06,"POLYGON ((172.89281 -40.50518, 172.73306 -40.5..."


In [36]:
sitename = "nzd0001"
inputs = {
    "polygon": list(poly.geometry[sitename].exterior.coords),
    "dates": ['2021-09-22', '2025-12-12'],
    "sat_list": ['L5','L7','L8'],
    "sitename": sitename,
    "filepath": 'data',
    "landsat_collection": 'C02',
}
result = SDS_download.check_images_available(inputs)

Number of images available between 2021-09-22 and 2025-12-12:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
     L5: 0 images
     L7: 2 images
     L8: 53 images
  Total images available to download from Tier 1: 55 images
  Total images available to download from Tier 2: 0 images


In [39]:
metadata = SDS_download.retrieve_images(inputs)

Number of images available between 2021-09-22 and 2025-12-12:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
     L5: 0 images
     L7: 2 images
     L8: 53 images
L5:There are 0 images available, 0 images already exist, 0 to download
L7:There are 2 images available, 0 images already exist, 2 to download
L8:There are 53 images available, 0 images already exist, 53 to download
  Total images available to download from Tier 1: 55 images
  Total images available to download from Tier 2: 0 images


nzd0001: Downloading Imagery for 3 satellites:   0%|          | 0/3 [00:00<?, ?it/s]

nzd0001: No images to download for L5


nzd0001: Downloading Imagery for L7:   0%|          | 0/2 [00:00<?, ?it/s]

skipping image '2023-01-04-20-25-07_L7_nzd0001_ms.tif' due to cloud and no data cover 100.00% exceeds threshold of 95.00%
skipping image '2023-03-01-20-19-07_L7_nzd0001_ms.tif' due to cloud and no data cover 100.00% exceeds threshold of 95.00%


nzd0001: Downloading Imagery for L8:   0%|          | 0/53 [00:00<?, ?it/s]

skipping image '2021-10-08-22-17-59_L8_nzd0001_ms.tif' due to cloud and no data cover 100.00% exceeds threshold of 95.00%
